In [ ]:
# %%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

fatal: destination path 'IndicTrans2' already exists and is not an empty directory.


In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
# %%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Cloning into 'IndicTransToolkit'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 218 (delta 59), reused 98 (delta 47), pack-reused 95 (from 1)
Receiving objects: 100% (218/218), 4.38 MiB | 8.08 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/IndicTransToolkit
Obtaining file:///content/IndicTransToolkit
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-q2f81ty4/indic-nlp-library-it2_a1d6f4aea6d0435bbc90fc0678a37be1
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-q2f81ty4/indic-nlp-library-it2_a1d6f4aea6d0435bbc90fc0678a37be1
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 342a7e95735e88949528bdca371518c

In [ ]:

import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:

def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-1B
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "eng_Latn", "doi_Deva"

print("Enter an English word to translate to Kashmiri. Type 'Q' to quit.\n")

while True:
    en_word = input("English : ").strip()
    if en_word == "Q":
        print("Translation process ended.")
        break

    hi_translations = batch_translate([en_word], src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

    print(f"Kashmiri : {hi_translations[0]}")
    print("------------------------------------------------------------------------------------")


Enter an English word to translate to Kashmiri. Type 'Q' to quit.

English : told
Kashmiri : दस्सेआ 
------------------------------------------------------------------------------------
English : give
Kashmiri : देओ 
------------------------------------------------------------------------------------
English : ancestors
Kashmiri : बापस दे पुरखे 
------------------------------------------------------------------------------------
English : Invite
Kashmiri : साद्दा देओ 
------------------------------------------------------------------------------------
English : Which
Kashmiri : जेह्ड़ा 
------------------------------------------------------------------------------------
English : Q
Translation process ended.
